<a href="https://colab.research.google.com/github/leorr/AI/blob/main/Projeto_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINKS REFERÊNCIAS:

Classify Images Using Python & Machine Learning - https://www.youtube.com/watch?v=iGWbqhdjf2s

 CNN Com cachorros e gatos - https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/

 what-are-max-pooling-average-pooling-global-max-pooling https://github.com/christianversloot/machine-learning-articles/blob/main/what-are-max-pooling-average-pooling-global-max-pooling-and-global-average-pooling.md

 Global Max Pool https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool2D

# Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# CONSTANTES

In [2]:
batch_size = 32 # Exemplos utilizados para um determinado processo
num_classes = 3 # classe de imagens possui 2 classes [Cachorros e Gatos]
dropout = 0.5 #rever
epochs = 500 #rever
lr = 0.001 #rever
IMG_HEIGHT = 150
IMG_WIDTH = 150

# Use GPU (opcional)

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Load model
Usamos softmax pois se espera "uma resposta correta"
https://stats.stackexchange.com/questions/233658/softmax-vs-sigmoid-function-in-logistic-classifier

In [4]:
with tf.device('/device:GPU:0'):
    base_model = tf.keras.applications.MobileNet(
        weights='imagenet',
        include_top=True,
        dropout=dropout,
        classes=num_classes,
        pooling=max,
        classifier_activation="softmax")

    
    model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=base_model.output
    )
    model.summary

In [5]:
with tf.device('/device:GPU:0'):
    for l in model.layers:
      if l.name.split('_')[0] != 'dense':
        l.trainable=False
      else:
        l.trainable=True

In [6]:
with tf.device('/cpu:0'):
    drive.mount('/content/drive', force_remount=True)
    test_cat_files = os.scandir("/content/drive/MyDrive/Python/afhq/train/cat")
    print(len([1 for x in test_cat_files if x.is_file()]))
    test_dogs_files = os.scandir("/content/drive/MyDrive/Python/afhq/train/dog")
    print(len([1 for x in test_dogs_files if x.is_file()]))
    test_wild_files = os.scandir("/content/drive/MyDrive/Python/afhq/train/wild")
    print(len([1 for x in test_wild_files if x.is_file()]))

Mounted at /content/drive
2500
4739
4288


# Load Images

In [7]:
with tf.device('/device:GPU:0'):
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) #included in our dependencies
    test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input) #included in our dependencies
    
    train_generator = train_data_gen.flow_from_directory('drive/MyDrive/Python/afhq/train',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     batch_size=batch_size,
                                                     class_mode="categorical",
                                                     shuffle=True)
    test_generator = test_data_gen.flow_from_directory('drive/MyDrive/Python/afhq/val',
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     batch_size=batch_size,
                                                     class_mode="categorical",
                                                     shuffle=True)
    total=train_generator.n+test_generator.n
    print(f"Porcentagem de data para teste: {((test_generator.n/total)*100):.2f}%")
    print(f"Porcentagem de data para treino: {((train_generator.n/total)*100):.2f}%")

Found 11527 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.
Porcentagem de data para teste: 11.51%
Porcentagem de data para treino: 88.49%


# Training model

In [8]:
with tf.device('/device:GPU:0'):
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

    step_size_train=train_generator.n//train_generator.batch_size
    step_size_test = test_generator.n//test_generator.batch_size

    model.fit_generator(generator=train_generator,
                       steps_per_epoch=step_size_train,
                       epochs=epochs,
                       validation_data=test_generator,
                       validation_steps=step_size_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/500
130/360 [=========>....................] - ETA: 29:20 - loss: 0.1689 - accuracy: 0.9363

KeyboardInterrupt: ignored